# Múltiplas Interfaces

Interfaces podem ser decompostas em interfaces menores que expressam subconjuntos de funcionalidades. Por exemplo, retomando a classe `DataSetComponent` do Zombie Health abaixo. Podemos pensar em duas interfaces para implementá-la: `IDataSource` que concentra funcionalidades relacionadas a qualquer fonte de dados; `ITableProducer` que concentra funcionalidades de qualquer objeto capaz de produzir dados tabulares (independentemente da origem desses dados).

Dividi-las em interfaces menores permite que uma classe não seja sempre obrigada a implementar tudo junto. Por exemplo, eu poderia ter um DataSource cujos dados sejam na forma de árvore, em vez de tabulares.

Veja abaixo como a classe `DataSetComponent` implementa as duas interfaces:

In [1]:
public interface IDataSource {
  public String getDataSource();
  public void setDataSource(String dataSource);
}

public interface ITableProducer {
  String[] requestAttributes();
  String[][] requestInstances();
}

In [2]:
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.util.ArrayList;

public class DataSetComponent implements IDataSource, ITableProducer {
  private String dataSource = null;
  private String[] attributes = null;
  private String[][] instances = null;
  
  public DataSetComponent() {
    /* nothing */
  }

  public String getDataSource() {
    return dataSource;
  }

  public void setDataSource(String dataSource) {
    this.dataSource = dataSource;
    if (dataSource == null) {
      attributes = null;
      instances = null;
    } else
      readDS();
  }
  
  public String[] requestAttributes() {
    return attributes;
  }
  
  public String[][] requestInstances() {
    return instances;
  }
  
  private void readDS() {
    ArrayList<String[]> instArray = new ArrayList<String[]>();
    try {
      BufferedReader file = new BufferedReader(new FileReader(dataSource));
        
      String line = file.readLine();
      if (line != null) {
        attributes = line.split(",");
        line = file.readLine();
        while (line != null) {
          String[] instLine = line.split(",");
          instArray.add(instLine);
          line = file.readLine();
        }
        instances = instArray.toArray(new String[0][]);
      }
        
      file.close();
    } catch (IOException erro) {
      erro.printStackTrace();
    }
  }
  
}

## Usando múltiplas interfaces

O problema de múltiplas interfaces é na hora de usar, já que você tem que indicar qual a interface usará a cada momento. Veja abaixo como fica o código.

Primeiro iniciou-se com a interface `IDataSource`, depois foi necessário mudar a referência para o objeto com a interface `ITableProducer`. Note que foi necessário um cast:
~~~java
ITableProducer tp = (ITableProducer)ds;
~~~

In [3]:
IDataSource ds = new DataSetComponent();
ds.setDataSource("../../../db/zombie/zombie-health-spreadsheet-ml-training.csv");

// aqui eh necessario se mudar de interface para se ter acesso aos metodos da tabela
ITableProducer tp = (ITableProducer)ds;

System.out.println("=== Attributes ===");
String attributes[] = tp.requestAttributes();
for (int a = 0; a < attributes.length-1; a++)
  System.out.print(attributes[a] + ", ");
System.out.println(attributes[attributes.length-1]);

System.out.println();
System.out.println("=== Instances ===");
String instances[][] = tp.requestInstances();
for (int i = 0; i < instances.length; i++) {
  for (int a = 0; a < attributes.length-1; a ++)
    System.out.print(instances[i][a] + ", ");
  System.out.println(instances[i][attributes.length-1]);
}

=== Attributes ===
paralysis, yellow_tong, member_loss, chest_pain, trembling_finger, severe_anger, history_bacteria, diagnostic

=== Instances ===
t, t, f, f, f, f, f, bacterial_infection
f, t, f, f, f, f, f, bacterial_infection
f, t, f, f, t, f, t, bite_deficit
f, t, t, f, t, f, f, bite_deficit
f, f, t, t, f, f, f, viral_infection
f, f, t, f, f, t, f, fights
f, f, f, f, f, t, f, nothing
f, f, f, f, t, f, f, bite_deficit
f, t, f, t, f, f, f, bacterial_infection
f, f, f, t, f, f, f, viral_infection
f, t, t, f, f, f, t, bite_deficit
t, t, f, f, f, f, f, bacterial_infection
f, f, f, t, f, f, t, viral_infection
f, f, t, f, f, f, f, fights
f, t, f, f, t, f, t, bite_deficit
f, t, t, f, t, f, f, bite_deficit
f, f, f, t, f, f, f, fights
f, t, f, f, f, f, f, bacterial_infection
f, f, f, f, t, f, f, bite_deficit


# Herança de Interfaces

Interfaces podem ser herdeiras de outras interfaces, ou seja, elas herdam a declaração de métodos da superinterface. Interfaces aceitam herança múltipla, então podemos tornar a solução anterior mais elegante criando uma interface que junta as duas chamada `IDataSet`, como segue:

In [4]:
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.util.ArrayList;

public interface IDataSet extends IDataSource, ITableProducer {
}

public class DataSetComponentII implements IDataSet {
  private String dataSource = null;
  private String[] attributes = null;
  private String[][] instances = null;
  
  public DataSetComponentII() {
    /* nothing */
  }

  public String getDataSource() {
    return dataSource;
  }

  public void setDataSource(String dataSource) {
    this.dataSource = dataSource;
    if (dataSource == null) {
      attributes = null;
      instances = null;
    } else
      readDS();
  }
  
  public String[] requestAttributes() {
    return attributes;
  }
  
  public String[][] requestInstances() {
    return instances;
  }
  
  private void readDS() {
    ArrayList<String[]> instArray = new ArrayList<String[]>();
    try {
      BufferedReader file = new BufferedReader(new FileReader(dataSource));
        
      String line = file.readLine();
      if (line != null) {
        attributes = line.split(",");
        line = file.readLine();
        while (line != null) {
          String[] instLine = line.split(",");
          instArray.add(instLine);
          line = file.readLine();
        }
        instances = instArray.toArray(new String[0][]);
      }
        
      file.close();
    } catch (IOException erro) {
      erro.printStackTrace();
    }
  }
  
}

## Usando a interface `IDataSet`

Como você verá a seguir, torna-se possível se resolver tudo com uma única interface:

In [5]:
IDataSet ds = new DataSetComponentII();
ds.setDataSource("../../../db/zombie/zombie-health-spreadsheet-ml-training.csv");

System.out.println("=== Attributes ===");
String attributes[] = ds.requestAttributes();
for (int a = 0; a < attributes.length-1; a++)
  System.out.print(attributes[a] + ", ");
System.out.println(attributes[attributes.length-1]);

System.out.println();
System.out.println("=== Instances ===");
String instances[][] = ds.requestInstances();
for (int i = 0; i < instances.length; i++) {
  for (int a = 0; a < attributes.length-1; a ++)
    System.out.print(instances[i][a] + ", ");
  System.out.println(instances[i][attributes.length-1]);
}

=== Attributes ===
paralysis, yellow_tong, member_loss, chest_pain, trembling_finger, severe_anger, history_bacteria, diagnostic

=== Instances ===
t, t, f, f, f, f, f, bacterial_infection
f, t, f, f, f, f, f, bacterial_infection
f, t, f, f, t, f, t, bite_deficit
f, t, t, f, t, f, f, bite_deficit
f, f, t, t, f, f, f, viral_infection
f, f, t, f, f, t, f, fights
f, f, f, f, f, t, f, nothing
f, f, f, f, t, f, f, bite_deficit
f, t, f, t, f, f, f, bacterial_infection
f, f, f, t, f, f, f, viral_infection
f, t, t, f, f, f, t, bite_deficit
t, t, f, f, f, f, f, bacterial_infection
f, f, f, t, f, f, t, viral_infection
f, f, t, f, f, f, f, fights
f, t, f, f, t, f, t, bite_deficit
f, t, t, f, t, f, f, bite_deficit
f, f, f, t, f, f, f, fights
f, t, f, f, f, f, f, bacterial_infection
f, f, f, f, t, f, f, bite_deficit
